<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Entrada-Antes-das-8h" data-toc-modified-id="Entrada-Antes-das-8h-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Entrada Antes das 8h</a></span></li><li><span><a href="#Ajustes-Manuais" data-toc-modified-id="Ajustes-Manuais-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Ajustes Manuais</a></span></li><li><span><a href="#Intervalo-no-Início-ou-Final-da-Jornada" data-toc-modified-id="Intervalo-no-Início-ou-Final-da-Jornada-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Intervalo no Início ou Final da Jornada</a></span></li><li><span><a href="#Intervalo-Cravado" data-toc-modified-id="Intervalo-Cravado-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Intervalo Cravado</a></span></li><li><span><a href="#Horário-Britânico" data-toc-modified-id="Horário-Britânico-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Horário Britânico</a></span></li></ul></div>

In [1]:
# Importando bibliotecas
import pandas as pd
import os
import time
from datetime import datetime, timedelta, date
from dateutil.relativedelta import relativedelta
import shutil
from warnings import filterwarnings
filterwarnings('ignore')

In [2]:
def time_to_datetime(df, date_col, time_col, prefix='ponto_', sufix='', drop=False):
    """
    Aplicação
    ----------

    
    Parâmetros
    ----------

    
    Retorno
    ----------

    
    Dependências
    ----------

    """
    
    # Cria coluna com concatenação entre data e tempo
    try:
        df[prefix + time_col + sufix] = df[date_col] + ' ' + df[time_col]
    except TypeError:
        # Uma ou mais colunas (date_col e time_col) não são strings
        df[prefix + time_col + sufix] = df[date_col].astype(str) + ' ' + df[time_col].astype(str)
    
    # Transforma em datetime
    df[prefix + time_col + sufix] = pd.to_datetime(df[prefix + time_col + sufix])
    
    # Valida drop da coluna antiga
    if drop & (prefix != '') & (sufix != ''):
        return df.drop(time_col, axis=1)
    else:
        return df

In [3]:
def valida_dados_origem(path, filename, data_mod=True, verbose=False):
    """
    Aplicação
    ----------
    Função para verificar se um determinado arquivo encontra-se em um determinado diretório origem.
    Como parâmetro adicional, pode-se validar a atualização do mesmo a partir do mês de modificação.
    
    Parâmetros
    ----------
    path: diretório completo da origem na qual será realizada a busca e validação
    filename: nome do arquivo (incluindo extensão) a ser procurado e validado na origem
    
    Retorno
    bool: variável booleana contendo validação de presença e data de modificação do arquivo
    
    Dependências
    bibliotecas: os, time, datetime
    """
    
    # Valida existência de arquivo na origem
    files_in_origin = os.listdir(path)
    if filename in files_in_origin:
        
        # Retorna parâmetros de modificação do arquivo
        mod_date = os.path.getmtime(path + '/' + filename)
        anomes_file = time.strftime('%Y%m', time.localtime(mod_date))
        anomes_atu = datetime.now().strftime('%Y%m')
        
        # Valida data de modificação com mês de atualização
        if anomes_atu == anomes_file:
            if verbose:
                print(f'Arquivo presente na origem e com data atualizada!')
            return True
        else:
            if verbose:
                print(f'[ALERTA] Arquivo presente na origem, porém destualizado! Leitura interrompida.')
                print(f'\nMês atual: {anomes_atu:>19}\nAtualização do arquivo: {anomes_file}')
            return False
    else:
        # Arquivo não presente na origem
        if verbose:
            print(f'[ALERTA] Arquivo {filename} não presente na origem ({path})')
            print(f'\nConteúdo do diretório: \n{files_in_origin}')
        return False

In [4]:
def copia_arquivo(origem, destino, verbose=False):
    """
    Aplicação
    ----------
    Copia um arquivo dos sistema operacional para um determinado destino
    
    Parâmetros
    ----------
    origem: caminho completo da origem + filename com extensão do arquivo a ser copiado
    destsino: caminho completo do destino + filename com extensão do arquivo copiado
    
    Retorno
    ----------
    None
    
    Dependências
    ----------
    bibliotecas: shutil
    """
    
    shutil.copyfile(origem, destino)

In [5]:
# Inicializando variáveis de controle
src_path = r'D:\Users\thiagoPanini\github_files\programming_languages\bank-stuff\gt-trab\data\raw_origin'
dst_path = r'D:\Users\thiagoPanini\github_files\programming_languages\bank-stuff\gt-trab\data'
ponto_filename = 'RANDOM_PONTO.csv'
src_datapath = os.path.join(src_path, ponto_filename)
dst_datapath = os.path.join(dst_path, ponto_filename)

# Validando arquivo de ponto presente na origem
if valida_dados_origem(path=src_path, filename=ponto_filename):
    
    # Copia arquivo da origem ao destino do projeto
    copia_arquivo(origem=src_datapath, destino=dst_datapath)
    
# Lendo dados
raw_ponto = pd.read_csv(dst_datapath)
print(f'Volumetria base original: {raw_ponto.shape}')

# Filtrando mês atual
anomes_atu = int((datetime.now() - relativedelta(months=1)).strftime('%Y%m'))
df_ponto = raw_ponto.copy()
df_ponto = df_ponto.query('mesref == @anomes_atu')
print(f'\nVolumetria base filtrada mês atual: {df_ponto.shape}')
df_ponto.sort_values(by=['numfunc', 'dia'], inplace=True)
df_ponto.head()

Volumetria base original: (100000, 12)

Volumetria base filtrada mês atual: (49652, 12)


,mesref,numfunc,racf,dia,dia_baliza,hora,desc_tipo_marcacao,desc_tipo_registro,deptid,database,dt_venc_acordo,codigo
2225,202008,1,ABCDEF,2020-08-01,2020-08-01,09:53,Saída,Incluído,803550957,2020-08-21,2020-08-21,TP1021
7698,202008,1,ABCDEF,2020-08-01,2020-08-01,10:51,Entrada,Original,189130830,2020-08-21,2020-08-21,TP1021
8337,202008,1,ABCDEF,2020-08-01,2020-08-01,17:03,Entrada,Original,400314106,2020-08-21,2020-08-21,TP1021
8535,202008,1,ABCDEF,2020-08-01,2020-08-01,14:58,Saída,Original,364202007,2020-08-21,2020-08-21,TP1021
10148,202008,1,ABCDEF,2020-08-01,2020-08-01,13:07,Saída,Incluído,754475693,2020-08-21,2020-08-21,TP1021


In [6]:
# Tipos primitivos originais
df_ponto.dtypes

mesref                 int64
numfunc                int64
racf                  object
dia                   object
dia_baliza            object
hora                  object
desc_tipo_marcacao    object
desc_tipo_registro    object
deptid                 int64
database              object
dt_venc_acordo        object
codigo                object
dtype: object

In [7]:
# Transformando tipos primitivos
date_cols = ['dia', 'dia_baliza', 'database', 'dt_venc_acordo']
for col in date_cols:
    df_ponto[col] = pd.to_datetime(df_ponto[col])

# Convertendo coluna de hora
df_ponto = time_to_datetime(df=df_ponto, date_col='dia', time_col='hora', 
                            prefix='', sufix='_ponto', drop=True)

# Novos tipos primitivos
df_ponto.dtypes

mesref                         int64
numfunc                        int64
racf                          object
dia                   datetime64[ns]
dia_baliza            datetime64[ns]
hora                          object
desc_tipo_marcacao            object
desc_tipo_registro            object
deptid                         int64
database              datetime64[ns]
dt_venc_acordo        datetime64[ns]
codigo                        object
hora_ponto            datetime64[ns]
dtype: object

In [8]:
df_ponto.head()

,mesref,numfunc,racf,dia,dia_baliza,hora,desc_tipo_marcacao,desc_tipo_registro,deptid,database,dt_venc_acordo,codigo,hora_ponto
2225,202008,1,ABCDEF,2020-08-01,2020-08-01,09:53,Saída,Incluído,803550957,2020-08-21,2020-08-21,TP1021,2020-08-01 09:53:00
7698,202008,1,ABCDEF,2020-08-01,2020-08-01,10:51,Entrada,Original,189130830,2020-08-21,2020-08-21,TP1021,2020-08-01 10:51:00
8337,202008,1,ABCDEF,2020-08-01,2020-08-01,17:03,Entrada,Original,400314106,2020-08-21,2020-08-21,TP1021,2020-08-01 17:03:00
8535,202008,1,ABCDEF,2020-08-01,2020-08-01,14:58,Saída,Original,364202007,2020-08-21,2020-08-21,TP1021,2020-08-01 14:58:00
10148,202008,1,ABCDEF,2020-08-01,2020-08-01,13:07,Saída,Incluído,754475693,2020-08-21,2020-08-21,TP1021,2020-08-01 13:07:00


In [9]:
# Definindo colunas a serem filtradas para armazenamento dos dados
cols_indicadores = ['numfunc', 'data', 'hora', 'tipo_marcacao', 'tipo_registro', 'tipo_ocorrencia']
cols_ponto = ['numfunc', 'dia', 'hora_ponto', 'desc_tipo_marcacao', 'desc_tipo_registro', 'tipo_ocorrencia']

# Criando DataFrame de indicadores para armazenar funcionais e ocorrências
df_indicadores = pd.DataFrame(columns=cols_indicadores)
df_indicadores

,numfunc,data,hora,tipo_marcacao,tipo_registro,tipo_ocorrencia


### Entrada Antes das 8h

_Escopo:_ Recorrência de marcações de entrada antes das 8h em um mesmo mês

* **Origens:** PONTO
* **Atributos:** `numfunc`, `hora`, `desc_tipo_marcacao`.
* **Procedimento:**
    - Criar flag para marcações de entrada (`desc_tipo_marcacao == 'Entrada'`)
    - Criar flag para marcações antes das 8h (`hora.hour < 8`)
    - Multiplicar os flags para filtrar entradas antes das 8h

In [10]:
def ind_entrada_pre_8h(df_indicadores, cols_ponto=cols_ponto, cols_indicadores=cols_indicadores):
    """
    Aplicação
    ----------

    
    Parâmetros
    ----------

    
    Retorno
    ----------

    
    Dependências
    ----------

    """
    
    # Criando marcação para o indicador
    mask = (df_ponto['desc_tipo_marcacao'] == 'Entrada') & df_ponto['hora_ponto'].apply(lambda x: x.hour < 8)
    df_ponto['entrada_pre_8h'] = mask * 1
    
    # Criando base consolidada de indicadores
    temp_indicador = df_ponto[mask]
    temp_indicador['tipo_ocorrencia'] = 'Entrada antes das 8h'
    temp_indicador = temp_indicador.loc[:, cols_ponto]
    temp_indicador.columns = cols_indicadores
    
    # Append na base de indicadores
    df_indicadores = df_indicadores.append(temp_indicador)
    
    return df_ponto, df_indicadores

In [11]:
# Executando função
df_indicadores = pd.DataFrame(columns=cols_indicadores)
df_ponto, df_indicadores = ind_entrada_pre_8h(df_indicadores)

# Verificando volumetrias
print(f'Volumetria da base de marcação de ponto: {df_ponto.shape}')
print(f'Volumetria da base de indicadores: {df_indicadores.shape}')
df_indicadores.head()

Volumetria da base de marcação de ponto: (49652, 14)
Volumetria da base de indicadores: (2129, 6)


,numfunc,data,hora,tipo_marcacao,tipo_registro,tipo_ocorrencia
36015,1,2020-08-01,2020-08-01 07:47:00,Entrada,Original,Entrada antes das 8h
35251,1,2020-08-02,2020-08-02 07:12:00,Entrada,Incluído,Entrada antes das 8h
92746,1,2020-08-02,2020-08-02 07:58:00,Entrada,Incluído,Entrada antes das 8h
45728,1,2020-08-04,2020-08-04 07:59:00,Entrada,Original,Entrada antes das 8h
79628,1,2020-08-06,2020-08-06 07:48:00,Entrada,Original,Entrada antes das 8h


### Ajustes Manuais

_Escopo:_ Recorrência de marcações incluídas em um mês

* **Origens:** PONTO
* **Atributos:** `numfunc`, `hora`, `desc_tipo_registro`.
* **Procedimento:**
     - Filtrar marcações de entrada na base origem (`desc_tipo_registro == 'Incluído'`)
     - Agrupar dados por numfunc e dia

In [12]:
def ind_ajustes_manuais(df_indicadores, cols_ponto=cols_ponto, cols_indicadores=cols_indicadores):
    """
    Aplicação
    ----------

    
    Parâmetros
    ----------

    
    Retorno
    ----------

    
    Dependências
    ----------

    """
    
    # Criando marcação para o indicador
    mask = df_ponto['desc_tipo_registro'] == 'Incluído'
    df_ponto['ajuste_manual'] = mask * 1
    
    # Criando base consolidada de indicadores
    temp_indicador = df_ponto[mask]
    temp_indicador['tipo_ocorrencia'] = 'Ajuste manual'
    temp_indicador = temp_indicador.loc[:, cols_ponto]
    temp_indicador.columns = cols_indicadores
    
    # Append na base de indicadores
    df_indicadores = df_indicadores.append(temp_indicador)
    
    return df_ponto, df_indicadores

In [13]:
# Executando função
df_ponto, df_indicadores = ind_ajustes_manuais(df_indicadores)

# Verificando volumetrias
print(f'Volumetria da base de marcação de ponto: {df_ponto.shape}')
print(f'Volumetria da base de indicadores: {df_indicadores.shape}')

Volumetria da base de marcação de ponto: (49652, 15)
Volumetria da base de indicadores: (26998, 6)


In [14]:
# Verificando base de ponto
df_ponto[(df_ponto['ajuste_manual'] == 1) & (df_ponto['entrada_pre_8h'] == 1)].head()

,mesref,numfunc,racf,dia,dia_baliza,hora,desc_tipo_marcacao,desc_tipo_registro,deptid,database,dt_venc_acordo,codigo,hora_ponto,entrada_pre_8h,ajuste_manual
35251,202008,1,ABCDEF,2020-08-02,2020-08-02,07:12,Entrada,Incluído,663215648,2020-08-21,2020-08-21,TP1021,2020-08-02 07:12:00,1,1
92746,202008,1,ABCDEF,2020-08-02,2020-08-02,07:58,Entrada,Incluído,827972756,2020-08-21,2020-08-21,TP1021,2020-08-02 07:58:00,1,1
815,202008,1,ABCDEF,2020-08-08,2020-08-08,07:21,Entrada,Incluído,648285837,2020-08-21,2020-08-21,TP1021,2020-08-08 07:21:00,1,1
97460,202008,1,ABCDEF,2020-08-08,2020-08-08,07:51,Entrada,Incluído,247038855,2020-08-21,2020-08-21,TP1021,2020-08-08 07:51:00,1,1
83875,202008,1,ABCDEF,2020-08-09,2020-08-09,07:38,Entrada,Incluído,121479489,2020-08-21,2020-08-21,TP1021,2020-08-09 07:38:00,1,1


In [15]:
# Verificando base de indicadores para 1 funcional em 1 dia
df_indicadores[(df_indicadores['numfunc'] == 1) & (df_indicadores['data'] == '2020-08-02')]

,numfunc,data,hora,tipo_marcacao,tipo_registro,tipo_ocorrencia
35251,1,2020-08-02,2020-08-02 07:12:00,Entrada,Incluído,Entrada antes das 8h
92746,1,2020-08-02,2020-08-02 07:58:00,Entrada,Incluído,Entrada antes das 8h
17396,1,2020-08-02,2020-08-02 18:01:00,Entrada,Incluído,Ajuste manual
27412,1,2020-08-02,2020-08-02 09:30:00,Entrada,Incluído,Ajuste manual
35251,1,2020-08-02,2020-08-02 07:12:00,Entrada,Incluído,Ajuste manual
56853,1,2020-08-02,2020-08-02 14:47:00,Entrada,Incluído,Ajuste manual
67018,1,2020-08-02,2020-08-02 08:13:00,Entrada,Incluído,Ajuste manual
71864,1,2020-08-02,2020-08-02 16:34:00,Saída,Incluído,Ajuste manual
75645,1,2020-08-02,2020-08-02 14:23:00,Saída,Incluído,Ajuste manual
92746,1,2020-08-02,2020-08-02 07:58:00,Entrada,Incluído,Ajuste manual


### Intervalo no Início ou Final da Jornada

_Escopo:_ Recorrência de marcações de intervalo no início ou no final da jornada.

* **Origens:** PONTO
* **Atributos:** `numfunc`, `hora`, `desc_tipo_marcacao`.
* **Procedimento:**
     - Adicionar um atributo na base considerando marcações "Expediente" e "Intervalo"
     - Medir diferença entre marcações "Intervalo" com marcações "Expediente"
     - Marcar ponto de marcações realizadas em intervalos curtos de tempo entre "Intervalo" e "Expediente"
* **Observações:**
    - Se marcações de intervalo ocorrerem entre 11:30h e 14:00h, a regra NÃO deve ser aplicada
    - Cargos comissionados (480 min no dia) tem limite de 2h de marcações para intervalo
    - Cargos não-comissionados (360 min no dia) tem limite de 1h de marcações para intervalo

**PROBLEMA: TEM QUE AGRUPAR POR DIA PRA VER AS HORAS MÍNIMA E MÁXIMA EM CADA DIA**

### Intervalo Cravado

_Escopo:_ Recorrência de marcações cravadas de intervalo de exatos 30 ou 60 minutos.

* **Origens:** PONTO
* **Atributos:** `numfunc`, `hora`, `desc_tipo_marcacao`.
* **Procedimento:**
     - Adicionar um atributo na base considerando marcações "Expediente" e "Intervalo"
     - Medir diferença entre marcações "Intervalo" com marcações "Expediente"
     - Marcar ponto de marcações realizadas em intervalos curtos de tempo entre "Intervalo" e "Expediente"
* **Observações:**
    - Se marcações de intervalo ocorrerem entre 11:30h e 14:00h, a regra NÃO deve ser aplicada
    - Cargos comissionados (480 min no dia) tem limite de 2h de marcações para intervalo
    - Cargos não-comissionados (360 min no dia) tem limite de 1h de marcações para intervalo

### Horário Britânico

In [16]:
df_ponto.head()

,mesref,numfunc,racf,dia,dia_baliza,hora,desc_tipo_marcacao,desc_tipo_registro,deptid,database,dt_venc_acordo,codigo,hora_ponto,entrada_pre_8h,ajuste_manual
2225,202008,1,ABCDEF,2020-08-01,2020-08-01,09:53,Saída,Incluído,803550957,2020-08-21,2020-08-21,TP1021,2020-08-01 09:53:00,0,1
7698,202008,1,ABCDEF,2020-08-01,2020-08-01,10:51,Entrada,Original,189130830,2020-08-21,2020-08-21,TP1021,2020-08-01 10:51:00,0,0
8337,202008,1,ABCDEF,2020-08-01,2020-08-01,17:03,Entrada,Original,400314106,2020-08-21,2020-08-21,TP1021,2020-08-01 17:03:00,0,0
8535,202008,1,ABCDEF,2020-08-01,2020-08-01,14:58,Saída,Original,364202007,2020-08-21,2020-08-21,TP1021,2020-08-01 14:58:00,0,0
10148,202008,1,ABCDEF,2020-08-01,2020-08-01,13:07,Saída,Incluído,754475693,2020-08-21,2020-08-21,TP1021,2020-08-01 13:07:00,0,1


In [17]:
ponto_group = df_ponto.groupby(by=['numfunc', 'hora', 'desc_tipo_marcacao'], as_index=False).count()
ponto_group = ponto_group.iloc[:, :4]
ponto_group.columns = ['numfunc', 'hora', 'desc_tipo_marcacao', 'qtd']
ponto_group.sort_values(by='qtd', ascending=False, inplace=True)
ponto_group = ponto_group.query('qtd > 1')
ponto_group.head(10)

,numfunc,hora,desc_tipo_marcacao,qtd
31072,74,17:06,Entrada,5
28532,68,17:12,Entrada,5
25002,60,11:26,Saída,5
10758,26,15:08,Entrada,5
33606,80,17:16,Saída,5
27847,67,09:14,Entrada,5
4025,10,12:40,Saída,5
14511,35,12:19,Saída,4
32192,77,13:13,Saída,4
35718,85,16:45,Entrada,4


In [18]:
df_ponto[(df_ponto['numfunc']==74) & (df_ponto['hora']=='17:06') & (df_ponto['desc_tipo_marcacao']=="Entrada")]

,mesref,numfunc,racf,dia,dia_baliza,hora,desc_tipo_marcacao,desc_tipo_registro,deptid,database,dt_venc_acordo,codigo,hora_ponto,entrada_pre_8h,ajuste_manual
40063,202008,74,ABCDEF,2020-08-19,2020-08-19,17:06,Entrada,Original,16391007,2020-08-21,2020-08-21,TP1021,2020-08-19 17:06:00,0,0
77803,202008,74,ABCDEF,2020-08-19,2020-08-19,17:06,Entrada,Original,936355528,2020-08-21,2020-08-21,TP1021,2020-08-19 17:06:00,0,0
6798,202008,74,ABCDEF,2020-08-23,2020-08-23,17:06,Entrada,Incluído,726370823,2020-08-21,2020-08-21,TP1021,2020-08-23 17:06:00,0,1
71444,202008,74,ABCDEF,2020-08-30,2020-08-30,17:06,Entrada,Original,821191236,2020-08-21,2020-08-21,TP1021,2020-08-30 17:06:00,0,0
80659,202008,74,ABCDEF,2020-08-30,2020-08-30,17:06,Entrada,Original,200853501,2020-08-21,2020-08-21,TP1021,2020-08-30 17:06:00,0,0


In [19]:
hora_brit = ponto_group.merge(df_ponto.loc[:, ['numfunc', 'dia', 'hora', 'hora_ponto', 'desc_tipo_marcacao', 
                                               'desc_tipo_registro']], how='left', on=['numfunc', 'hora', 
                                                                                       'desc_tipo_marcacao'])

In [20]:
hora_brit.head()

,numfunc,hora,desc_tipo_marcacao,qtd,dia,hora_ponto,desc_tipo_registro
0,74,17:06,Entrada,5,2020-08-19,2020-08-19 17:06:00,Original
1,74,17:06,Entrada,5,2020-08-19,2020-08-19 17:06:00,Original
2,74,17:06,Entrada,5,2020-08-23,2020-08-23 17:06:00,Incluído
3,74,17:06,Entrada,5,2020-08-30,2020-08-30 17:06:00,Original
4,74,17:06,Entrada,5,2020-08-30,2020-08-30 17:06:00,Original


In [21]:
df_ponto_teste = df_ponto.merge(ponto_group, how='left', on=['numfunc', 'hora', 'desc_tipo_marcacao'])
df_ponto_teste['horario_britanico'] = df_ponto_teste['qtd'].apply(lambda x: 1 if x > 1 else 0)
df_ponto_teste.drop('qtd', axis=1, inplace=True)
df_ponto_teste.head(10)

,mesref,numfunc,racf,dia,dia_baliza,hora,desc_tipo_marcacao,desc_tipo_registro,deptid,database,dt_venc_acordo,codigo,hora_ponto,entrada_pre_8h,ajuste_manual,horario_britanico
0,202008,1,ABCDEF,2020-08-01,2020-08-01,09:53,Saída,Incluído,803550957,2020-08-21,2020-08-21,TP1021,2020-08-01 09:53:00,0,1,0
1,202008,1,ABCDEF,2020-08-01,2020-08-01,10:51,Entrada,Original,189130830,2020-08-21,2020-08-21,TP1021,2020-08-01 10:51:00,0,0,0
2,202008,1,ABCDEF,2020-08-01,2020-08-01,17:03,Entrada,Original,400314106,2020-08-21,2020-08-21,TP1021,2020-08-01 17:03:00,0,0,0
3,202008,1,ABCDEF,2020-08-01,2020-08-01,14:58,Saída,Original,364202007,2020-08-21,2020-08-21,TP1021,2020-08-01 14:58:00,0,0,0
4,202008,1,ABCDEF,2020-08-01,2020-08-01,13:07,Saída,Incluído,754475693,2020-08-21,2020-08-21,TP1021,2020-08-01 13:07:00,0,1,1
5,202008,1,ABCDEF,2020-08-01,2020-08-01,10:18,Entrada,Original,980076855,2020-08-21,2020-08-21,TP1021,2020-08-01 10:18:00,0,0,0
6,202008,1,ABCDEF,2020-08-01,2020-08-01,11:14,Entrada,Incluído,544042144,2020-08-21,2020-08-21,TP1021,2020-08-01 11:14:00,0,1,1
7,202008,1,ABCDEF,2020-08-01,2020-08-01,16:07,Entrada,Original,905517208,2020-08-21,2020-08-21,TP1021,2020-08-01 16:07:00,0,0,0
8,202008,1,ABCDEF,2020-08-01,2020-08-01,17:51,Saída,Original,39116099,2020-08-21,2020-08-21,TP1021,2020-08-01 17:51:00,0,0,0
9,202008,1,ABCDEF,2020-08-01,2020-08-01,11:13,Entrada,Incluído,960843710,2020-08-21,2020-08-21,TP1021,2020-08-01 11:13:00,0,1,1


In [22]:
hora_brit.tail()

,numfunc,hora,desc_tipo_marcacao,qtd,dia,hora_ponto,desc_tipo_registro
14952,11,15:22,Saída,2,2020-08-14,2020-08-14 15:22:00,Incluído
14953,14,10:57,Entrada,2,2020-08-06,2020-08-06 10:57:00,Original
14954,14,10:57,Entrada,2,2020-08-22,2020-08-22 10:57:00,Original
14955,6,10:18,Saída,2,2020-08-15,2020-08-15 10:18:00,Original
14956,6,10:18,Saída,2,2020-08-27,2020-08-27 10:18:00,Incluído


In [23]:
len(df_ponto)

49652

In [24]:
len(hora_brit)

14957

In [25]:
df_indicadores.head()

,numfunc,data,hora,tipo_marcacao,tipo_registro,tipo_ocorrencia
36015,1,2020-08-01,2020-08-01 07:47:00,Entrada,Original,Entrada antes das 8h
35251,1,2020-08-02,2020-08-02 07:12:00,Entrada,Incluído,Entrada antes das 8h
92746,1,2020-08-02,2020-08-02 07:58:00,Entrada,Incluído,Entrada antes das 8h
45728,1,2020-08-04,2020-08-04 07:59:00,Entrada,Original,Entrada antes das 8h
79628,1,2020-08-06,2020-08-06 07:48:00,Entrada,Original,Entrada antes das 8h


In [26]:
def ind_horario_britanico(df_indicadores=df_indicadores, df_ponto=df_ponto, cols_ponto=cols_ponto, 
                          cols_indicadores=cols_indicadores):
    """
    Aplicação
    ----------

    
    Parâmetros
    ----------

    
    Retorno
    ----------

    
    Dependências
    ----------

    """
    
    # Agrupando dados por funcional, hora e tipo de marcação
    ponto_group = df_ponto.groupby(by=['numfunc', 'hora', 'desc_tipo_marcacao'], as_index=False).count()
    ponto_group = ponto_group.iloc[:, :4]
    ponto_group.columns = ['numfunc', 'hora', 'desc_tipo_marcacao', 'qtd']
    ponto_group.sort_values(by='qtd', ascending=False, inplace=True)
    ponto_group = ponto_group.query('qtd > 1')
    
    # Criando marcação para o indicador
    df_ponto = df_ponto.merge(ponto_group, how='left', on=['numfunc', 'hora', 'desc_tipo_marcacao'])
    df_ponto['horario_britanico'] = df_ponto['qtd'].apply(lambda x: 1 if x > 1 else 0)
    df_ponto.drop('qtd', axis=1, inplace=True)
    
    # Criando máskara
    mask = df_ponto['horario_britanico'] == 1
    
    # Criando base consolidada de indicadores
    temp_indicador = df_ponto[mask]
    temp_indicador['tipo_ocorrencia'] = 'Horário Britânico'
    temp_indicador = temp_indicador.loc[:, cols_ponto]
    temp_indicador.columns = cols_indicadores
    
    # Append na base de indicadores
    df_indicadores = df_indicadores.append(temp_indicador)
    
    return df_ponto, df_indicadores

In [27]:
# Executando função
df_ponto, df_indicadores = ind_horario_britanico()

# Verificando volumetrias
print(f'Volumetria da base de marcação de ponto: {df_ponto.shape}')
print(f'Volumetria da base de indicadores: {df_indicadores.shape}')

Volumetria da base de marcação de ponto: (49652, 16)
Volumetria da base de indicadores: (41955, 6)


In [28]:
# Validando
df_ponto[df_ponto['horario_britanico'] == 1].head()

,mesref,numfunc,racf,dia,dia_baliza,hora,desc_tipo_marcacao,desc_tipo_registro,deptid,database,dt_venc_acordo,codigo,hora_ponto,entrada_pre_8h,ajuste_manual,horario_britanico
4,202008,1,ABCDEF,2020-08-01,2020-08-01,13:07,Saída,Incluído,754475693,2020-08-21,2020-08-21,TP1021,2020-08-01 13:07:00,0,1,1
6,202008,1,ABCDEF,2020-08-01,2020-08-01,11:14,Entrada,Incluído,544042144,2020-08-21,2020-08-21,TP1021,2020-08-01 11:14:00,0,1,1
9,202008,1,ABCDEF,2020-08-01,2020-08-01,11:13,Entrada,Incluído,960843710,2020-08-21,2020-08-21,TP1021,2020-08-01 11:13:00,0,1,1
14,202008,1,ABCDEF,2020-08-01,2020-08-01,10:45,Entrada,Incluído,887667876,2020-08-21,2020-08-21,TP1021,2020-08-01 10:45:00,0,1,1
17,202008,1,ABCDEF,2020-08-01,2020-08-01,14:59,Saída,Incluído,875582518,2020-08-21,2020-08-21,TP1021,2020-08-01 14:59:00,0,1,1


In [29]:
df_indicadores[(df_indicadores['numfunc']==1) & (df_indicadores['data']=='2020-08-01')]

,numfunc,data,hora,tipo_marcacao,tipo_registro,tipo_ocorrencia
36015,1,2020-08-01,2020-08-01 07:47:00,Entrada,Original,Entrada antes das 8h
2225,1,2020-08-01,2020-08-01 09:53:00,Saída,Incluído,Ajuste manual
10148,1,2020-08-01,2020-08-01 13:07:00,Saída,Incluído,Ajuste manual
13171,1,2020-08-01,2020-08-01 11:14:00,Entrada,Incluído,Ajuste manual
16204,1,2020-08-01,2020-08-01 11:13:00,Entrada,Incluído,Ajuste manual
18573,1,2020-08-01,2020-08-01 09:18:00,Entrada,Incluído,Ajuste manual
29960,1,2020-08-01,2020-08-01 14:34:00,Entrada,Incluído,Ajuste manual
31513,1,2020-08-01,2020-08-01 10:45:00,Entrada,Incluído,Ajuste manual
37932,1,2020-08-01,2020-08-01 17:41:00,Saída,Incluído,Ajuste manual
41534,1,2020-08-01,2020-08-01 14:59:00,Saída,Incluído,Ajuste manual


In [30]:
df_indicadores[(df_indicadores['numfunc']==1) & (df_indicadores['data']=='2020-08-01')]['tipo_ocorrencia'].value_counts()

Ajuste manual           13
Horário Britânico        7
Entrada antes das 8h     1
Name: tipo_ocorrencia, dtype: int64

In [31]:
df_ponto[(df_ponto['numfunc']==1) & (df_ponto['dia']=='2020-08-01')]['entrada_pre_8h'].sum()

1

In [32]:
df_ponto[(df_ponto['numfunc']==1) & (df_ponto['dia']=='2020-08-01')]['ajuste_manual'].sum()

13

In [33]:
df_ponto[(df_ponto['numfunc']==1) & (df_ponto['dia']=='2020-08-01')]['horario_britanico'].sum()

7